In [16]:
import json
from unskript import nbparams
from unskript.secrets import ENV_MODE, ENV_MODE_AWS
from unskript.fwk.workflow import Task, Workflow

env = {"ENV_MODE":"ENV_MODE_AWS","TENANT_ID":"982dba5f-d9df-48ae-a5bf-ec1fc94d4882","ENVIRONMENT_ID":"1499f27c-6406-4fbd-bd1b-c6f92800018f","TENANT_URL":"https://tenant-staging.alpha.unskript.io","AWS_REGION":"us-west-2"}
secret_store_cfg = {"SECRET_STORE_TYPE":"SECRET_STORE_TYPE_AWS","AWS_SECRET_PREFIX":"staging","AWS_REGION":"us-west-2"}
paramDict = {"tag_key": "Service", "tag_value": "devmongodb", "user_name": "Test"}
paramDict.update(env)
paramDict.update(secret_store_cfg)
paramsJson = json.dumps(paramDict)
nbParamsObj = nbparams.NBParams(paramsJson)
tag_key = nbParamsObj.get('tag_key')
tag_value = nbParamsObj.get('tag_value')
user_name = nbParamsObj.get('user_name')

w = Workflow(env, secret_store_cfg, None, global_vars=globals())

<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to add a new IAM user using unSkript legos.</b>
</div>

<br>

<center><h2>Add New IAM User</h2></center>

# Steps Overview
1) Created New IAM user by passing User name and tags. 
2) Created login profile for the newly created user.
3) Check the caller identity of the current user.
4) Send success IAM user creation message to the slack channel.

Here we will use unSkript Create IAM User Lego. This lego takes user_name, tag_key and tag_value as input. This inputs is used to create a new IAM user for AWS.

In [23]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from botocore.exceptions import ClientError
import pprint

from beartype import beartype
@beartype
def aws_create_iam_user_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_create_iam_user(handle, user_name: str, tag_key: str, tag_value: str) -> Dict:
    """aws_create_iam_user Creates new IAM User.

        :type user_name: string
        :param user_name: Name of new IAM User.

        :type tag_key: string
        :param tag_key: Tag Key to new User.

        :type tag_value: string
        :param tag_value: Tag Value to new User.

        :rtype: Dict with the stopped instances state info.
    """

    ec2Client = handle.client("iam")
    result = {}
    try:
        response = ec2Client.create_user(
            UserName=user_name,
            Tags=[
                {
                    'Key': tag_key,
                    'Value': tag_value
                }])
        result = response
    except ClientError as error:
        if error.response['Error']['Code'] == 'EntityAlreadyExists':
            result = error.response
        else:
            result = error.response

    return result


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "DevRole",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "0b438eba-0627-4f6d-b998-a4c604f20e3c"
}''')
task.configure(inputParamsJson='''{
    "tag_key": "tag_key",
    "tag_value": "tag_value",
    "user_name": "user_name"
    }''')
task.configure(outputName="UserInfo")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_create_iam_user, lego_printer=aws_create_iam_user_printer, hdl=hdl, args=args)

Here we will use unSkript Create Login Profile for IAM User Lego. This lego takes UserName and Password as input. This inputs is used to create a login profile for new IAM user.

In [24]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Dict
from botocore.exceptions import ClientError
import pprint

from beartype import beartype
@beartype
def aws_create_user_login_profile_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_create_user_login_profile(handle, UserName: str, Password: str) -> Dict:
    """aws_create_user_login_profile Create login profile for IAM User.

        :type UserName: string
        :param UserName: Name of new IAM User.

        :type Password: string
        :param Password: Tag Key to new User.

        :rtype: Dict with the Profile Creation status info.
    """

    ec2Client = handle.client("iam")
    result = {}
    try:
        response = ec2Client.create_login_profile(
            UserName=UserName,
            Password=Password,
            PasswordResetRequired=True)

        result = response
    except ClientError as error:
        if error.response['Error']['Code'] == 'EntityAlreadyExists':
            result = error.response
        else:
            result = error.response

    return result


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "DevRole",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "0b438eba-0627-4f6d-b998-a4c604f20e3c"
}''')
task.configure(inputParamsJson='''{
    "Password": "\\"test@123\\"",
    "UserName": "user_name"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "'User' in UserInfo",
    "condition_result": true
    }''')


(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_create_user_login_profile, lego_printer=aws_create_user_login_profile_printer, hdl=hdl, args=args)

Here we will use unSkript STS Get Caller Identity Lego. This lego did't take any inputs. Shows the the caller identity for the current user.

In [20]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
import pprint


from beartype import beartype
@beartype
def aws_get_caller_identity_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_get_caller_identity(handle) -> Dict:
    ec2Client = handle.client('sts')
    response = ec2Client.get_caller_identity()

    return response


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "DevRole",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "0b438eba-0627-4f6d-b998-a4c604f20e3c"
}''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_caller_identity, lego_printer=aws_get_caller_identity_printer, hdl=hdl, args=args)

Here we will use unSkript Post Slack Message Lego. This lego takes channel name and message as input. This inputs is used to to post the message when new IAM user created.

In [21]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint

from pydantic import BaseModel, Field
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

pp = pprint.PrettyPrinter(indent=2)


from beartype import beartype
@beartype
def slack_post_message_printer(output):
    if output is not None:
        pprint.pprint(output)
    else:
        return


@beartype
def slack_post_message(
        handle: WebClient,
        channel: str,
        message: str) -> str:

    try:
        response = handle.chat_postMessage(
            channel=channel,
            text=message)
        return f"Successfuly Sent Message on Channel: #{channel}"
    except SlackApiError as e:
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.response['error']}")
        if e.response['error'] == 'channel_not_found':
            raise Exception('Channel Not Found')
        elif e.response['error'] == 'duplicate_channel_not_found':
            raise Exception('Channel associated with the message_id not valid')
        elif e.response['error'] == 'not_in_channel':
            raise Exception('Cannot post message to channel user is not in')
        elif e.response['error'] == 'is_archived':
            raise Exception('Channel has been archived')
        elif e.response['error'] == 'msg_too_long':
            raise Exception('Message text is too long')
        elif e.response['error'] == 'no_text':
            raise Exception('Message text was not provided')
        elif e.response['error'] == 'restricted_action':
            raise Exception('Workspace preference prevents user from posting')
        elif e.response['error'] == 'restricted_action_read_only_channel':
            raise Exception('Cannot Post message, read-only channel')
        elif e.response['error'] == 'team_access_not_granted':
            raise Exception('The token used is not granted access to the workspace')
        elif e.response['error'] == 'not_authed':
            raise Exception('No Authtnecition token provided')
        elif e.response['error'] == 'invalid_auth':
            raise Exception('Some aspect of Authentication cannot be validated. Request denied')
        elif e.response['error'] == 'access_denied':
            raise Exception('Access to a resource specified in the request denied')
        elif e.response['error'] == 'account_inactive':
            raise Exception('Authentication token is for a deleted user')
        elif e.response['error'] == 'token_revoked':
            raise Exception('Authentication token for a deleted user has been revoked')
        elif e.response['error'] == 'no_permission':
            raise Exception('The workspace toekn used does not have necessary permission to send message')
        elif e.response['error'] == 'ratelimited':
            raise Exception('The request has been ratelimited. Retry sending message later')
        elif e.response['error'] == 'service_unavailable':
            raise Exception('The service is temporarily unavailable')
        elif e.response['error'] == 'fatal_error':
            raise Exception('The server encountered catostrophic error while sending message')
        elif e.response['error'] == 'internal_error':
            raise Exception('The server could not complete operation, likely due to transietn issue')
        elif e.response['error'] == 'request_timeout':
            raise Exception('Sending message error via POST: either message was missing or truncated')
        else:
            raise Exception(f'Failed Sending Message to slack channel {channel} Error: {e.response["error"]}')

    except Exception as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.__str__()}")
        return f"Unable to send message on {channel}"


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "unSkript",
    "credential_type": "CONNECTOR_TYPE_SLACK",
    "credential_id": "b1adafe4-6593-484d-8fc1-e40799378e9a"
}''')
task.configure(inputParamsJson='''{
    "channel": "\\"dummy\\"",
    "message": "\\"New IAM user {}\\".format(user_name)"
    }''')

task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "'User' in UserInfo",
    "condition_result": true
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(slack_post_message, lego_printer=slack_post_message_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's AWS and slack to perform AWS create new IAM user and login profile for the same user and also show the caller identity of the current user and post the message on the slack channel. To view the full platform capabilities of unSkript please visit https://unskript.com